In [55]:
from datetime import datetime
import json
from multiprocessing.pool import ThreadPool
import os
from typing import Any, Union
from urllib.parse import urlparse

import boto3
import fiona
from pystac import (
    Catalog,
    CatalogType,
    Collection,
    Extent,
    Link,
    MediaType,
    Provider,
    RelType,
    SpatialExtent,
    TemporalExtent
)
from pystac.stac_io import DefaultStacIO, StacIO
from rio_stac import create_stac_item

In [56]:
class CustomStacIO(DefaultStacIO):
    def __init__(self):
        self.s3 = boto3.resource("s3")

    def write_text(
        self, dest: Union[str, Link], txt: str, *args: Any, **kwargs: Any
    ) -> None:
        parsed = urlparse(dest)
        if parsed.scheme == "https":
            bucket = parsed.netloc.split(".")[0]
            key = parsed.path[1:]
            print(bucket, key)
            s3 = boto3.resource("s3")
            s3.Object(bucket, key).put(Body=txt, ContentEncoding="utf-8")
        else:
            super().write_text(dest, txt, *args, **kwargs)

StacIO.set_default(CustomStacIO)

# 1. Get a list of images

In [57]:
%%bash
git clone https://github.com/justinelliotmeyers/ESRI_2020_LULC_TILES

Cloning into 'ESRI_2020_LULC_TILES'...


In [58]:
file_list = []
with fiona.open("ESRI_2020_LULC_TILES/esri_2020_lulc.shp") as f:
    for feat in f:
        href = feat['properties']['DL']
        file_list.append(href)

# 2. Create the STAC Items

In [59]:
def _create_stac_item(filename):
    print("Processing: ", filename)
    item = create_stac_item(
        source=filename,
        properties={
            "start_datetime": "2020-01-01T00:00:00.00Z",
            "end_datetime": "2021-01-01T00:00:00.00Z"
        },
        id=os.path.splitext(os.path.split(filename)[-1])[0],
        asset_name="image",
        asset_href=filename,
        asset_media_type=MediaType.COG,
        with_proj=True,
        with_raster=True
    )

    # Add storage extension
    item.assets['image'].properties = {
        "storage:platform": "AWS",
        "storage:region": "us-east-1",
        "storage:request_pays": False,
        "storage:tier": "Standard"
    }
    item.stac_extensions.append("https://stac-extensions.github.io/storage/v1.0.0/schema.json",)
    item.validate()
    return item


### 2a. Transform a single item.

In [6]:
stac_item = _create_stac_item(file_list[0])
print(json.dumps(stac_item.to_dict(), indent=4))

Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/32S_20200101-20210101.tif
{
    "type": "Feature",
    "stac_version": "1.0.0",
    "id": "32S_20200101-20210101",
    "properties": {
        "start_datetime": "2020-01-01T00:00:00.00Z",
        "end_datetime": "2021-01-01T00:00:00.00Z",
        "proj:epsg": 32632,
        "proj:geometry": {
            "type": "Polygon",
            "coordinates": [
                [
                    [
                        128633.28570658935,
                        4490488.595753916
                    ],
                    [
                        128633.28570658935,
                        3490568.5957539156
                    ],
                    [
                        880093.2857065893,
                        3490568.5957539156
                    ],
                    [
                        880093.2857065893,
                        4490488.595753916
                    ],
                    [
          

### 2b Create all the items.

In [7]:
pool = ThreadPool()
stac_items = pool.map(_create_stac_item, file_list)

Processing: Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/44S_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/32U_20200101-20210101.tif https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/32S_20200101-20210101.tif

Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/47D_20200101-20210101.tifProcessing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/17N_20200101-20210101.tif

Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/17M_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/38L_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/29P_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/14C_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/38C_20200101-20210101.tif
Processing:  https:/

Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/50J_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/47V_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/11S_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/47T_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/26E_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/56L_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/38R_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/29N_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/41K_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/14P_20200101-20210101.tif
Processing:  https:/

Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/31S_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/59K_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/14T_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/20J_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/05U_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/50W_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/19H_20200101-20210101.tifProcessing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/05M_20200101-20210101.tif

Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/60H_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/20Q_20200101-20210101.tif
Processing:  https:/

Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/13S_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/54D_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/19L_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/15P_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/18M_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/55D_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/54S_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/40S_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/43N_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/52M_20200101-20210101.tif
Processing:  https:/

Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/49M_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/30C_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/46C_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/34L_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/21P_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/31M_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/46S_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/55C_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/13C_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/28P_20200101-20210101.tif
Processing:  https:/

Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/09C_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/55K_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/15Q_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/54J_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/42C_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/22C_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/33K_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/39P_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/21J_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/15T_20200101-20210101.tif
Processing:  https:/

Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/39V_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/16T_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/07V_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/46T_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/40R_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/37G_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/03U_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/16U_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/49K_20200101-20210101.tifProcessing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/27R_20200101-20210101.tif

Processing:  https:/

Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/48V_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/11V_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/27X_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/34Q_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/45R_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/42U_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/40X_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/51X_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/19E_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/49W_20200101-20210101.tif
Processing:  https:/

Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/20X_20200101-20210101.tif
Processing: Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/14X_20200101-20210101.tif
 https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/12V_20200101-20210101.tif
Processing:  Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/55X_20200101-20210101.tif
https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/37W_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/34P_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/35V_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/22X_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/12P_20200101-20210101.tif
Processing:  https://soilspackage-useast.s3.amazonaws.com/EsriLandCover/26X_20200101-20210101.tif
Processing:  https:/

# 3. Build the catalog.

In [35]:
CATALOG_S3_PREFIX = "https://esri-lulc-2020-stac.s3.amazonaws.com"

In [36]:
root_catalog = Catalog(
    id="esri-lulc-2020",
    description="A ten class global land use/land cover (LULC) map for the year 2020 at 10 meter resolution.  Created by Impact Observatory in partnership with Microsoft and ESRI.",
    title="esri-lulc-2020",
)
root_catalog.normalize_hrefs(CATALOG_S3_PREFIX)

### 3a. Add a collection

In [37]:
# Calculate extent
from shapely.geometry import shape, MultiPolygon
bounds = MultiPolygon(
    [shape(feat.geometry) for feat in stac_items]
).bounds


collection = Collection(
    id="esri-lulc-2020",
    description="A ten class global land use/land cover (LULC) map for the year 2020 at 10 meter resolution.  Created by Impact Observatory in partnership with Microsoft and ESRI.",
    title="esri-lulc-2020",
    license="CC-BY-4.0",
    extent=Extent(
        spatial=SpatialExtent(
            [bounds]
        ),
        temporal=TemporalExtent(
            [[datetime(year=2020, month=1, day=1), datetime(year=2021, month=1, day=1)]]
        )
    ),
    providers=[
        Provider(
            name="Impact Observatory",
            url="https://www.impactobservatory.com/",
            roles=["processor"]
        ),
        Provider(
            name="ESRI",
            url="https://esri.com",
            roles=["host", "licensor"]
        ),
        Provider(
            name="Microsoft",
            url="https://www.microsoft.com/",
            roles=["processor"]
        )
    ]
)
collection.add_link(
   Link(
        rel=RelType.LICENSE,
        target="https://creativecommons.org/licenses/by/4.0/",
        media_type="text/html"
    ), 
)
root_catalog.add_child(collection)

### 3a. Create a child catalog for each UTM zone to split up the data.

In [43]:
# Bin the items by epsg
utm_zones = {}
for stac_item in stac_items:
    epsg = stac_item.properties['proj:epsg']
    if epsg not in utm_zones.keys():
        utm_zones[epsg] = [stac_item]
    else:
        utm_zones[epsg].append(stac_item)

# Build the catalog
for (zone, items) in utm_zones.items():
    zone = str(zone)
    if zone[:2] == '326':
        orientation = 'N'
    else:
        orientation = 'S'
    number = zone[-2:]
    zone_id = f"UTM Zone {number}{orientation}"    
    
    utm_catalog = Catalog(
        id=zone,
        description=f"Landcover data projected in EPSG:{zone} ({zone_id})"
    )
    
    # Add the catalog to the parent collection
    collection.add_child(utm_catalog)
    
    # Add the items to the child catalog
    for item in items:
        utm_catalog.add_item(item)

root_catalog.describe(include_hrefs=True)

* <Catalog id=esri-lulc-2020> https://esri-lulc-2020-stac.s3.amazonaws.com/catalog.json
    * <Collection id=esri-lulc-2020> https://esri-lulc-2020-stac.s3.amazonaws.com/esri-lulc-2020/collection.json
        * <Catalog id=32632> https://esri-lulc-2020-stac.s3.amazonaws.com/esri-lulc-2020/32632/catalog.json
          * <Item id=32S_20200101-20210101> https://esri-lulc-2020-stac.s3.amazonaws.com/esri-lulc-2020/32632/32S_20200101-20210101/32S_20200101-20210101.json
          * <Item id=32R_20200101-20210101> https://esri-lulc-2020-stac.s3.amazonaws.com/esri-lulc-2020/32632/32R_20200101-20210101/32R_20200101-20210101.json
          * <Item id=32U_20200101-20210101> https://esri-lulc-2020-stac.s3.amazonaws.com/esri-lulc-2020/32632/32U_20200101-20210101/32U_20200101-20210101.json
          * <Item id=32Q_20200101-20210101> https://esri-lulc-2020-stac.s3.amazonaws.com/esri-lulc-2020/32632/32Q_20200101-20210101/32Q_20200101-20210101.json
          * <Item id=32P_20200101-20210101> https://esr

# 4. Save the catalog

In [39]:
root_catalog.validate_all()

In [53]:
root_catalog.save()

esri-lulc-2020-stac esri-lulc-2020/32632/32S_20200101-20210101/32S_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32632/32R_20200101-20210101/32R_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32632/32U_20200101-20210101/32U_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32632/32Q_20200101-20210101/32Q_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32632/32P_20200101-20210101/32P_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32632/32T_20200101-20210101/32T_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32632/32N_20200101-20210101/32N_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32632/32W_20200101-20210101/32W_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32632/32V_20200101-20210101/32V_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32632/catalog.json
esri-lulc-2020-stac esri-lulc-2020/32735/35H_20200101-20210101/35H_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32735/35D_2

esri-lulc-2020-stac esri-lulc-2020/32644/44S_20200101-20210101/44S_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32644/44N_20200101-20210101/44N_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32644/44P_20200101-20210101/44P_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32644/44Q_20200101-20210101/44Q_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32644/44R_20200101-20210101/44R_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32644/44T_20200101-20210101/44T_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32644/44V_20200101-20210101/44V_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32644/44U_20200101-20210101/44U_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32644/44W_20200101-20210101/44W_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32644/44X_20200101-20210101/44X_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32644/catalog.json
esri-lulc-2020-stac esri-lulc-2020/32647/47S_2

esri-lulc-2020-stac esri-lulc-2020/32629/29W_20200101-20210101/29W_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32629/29X_20200101-20210101/29X_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32629/catalog.json
esri-lulc-2020-stac esri-lulc-2020/32738/38D_20200101-20210101/38D_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32738/38M_20200101-20210101/38M_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32738/38L_20200101-20210101/38L_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32738/38C_20200101-20210101/38C_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32738/38K_20200101-20210101/38K_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32738/38J_20200101-20210101/38J_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32738/catalog.json
esri-lulc-2020-stac esri-lulc-2020/32741/41D_20200101-20210101/41D_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32741/41C_20200101-20210101/41C_20200101-202101

esri-lulc-2020-stac esri-lulc-2020/32722/22E_20200101-20210101/22E_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32722/catalog.json
esri-lulc-2020-stac esri-lulc-2020/32718/18H_20200101-20210101/18H_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32718/18D_20200101-20210101/18D_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32718/18M_20200101-20210101/18M_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32718/18L_20200101-20210101/18L_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32718/18C_20200101-20210101/18C_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32718/18K_20200101-20210101/18K_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32718/18G_20200101-20210101/18G_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32718/18F_20200101-20210101/18F_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32718/catalog.json
esri-lulc-2020-stac esri-lulc-2020/32721/21H_20200101-20210101/21H_20200101-202101

esri-lulc-2020-stac esri-lulc-2020/32610/10T_20200101-20210101/10T_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32610/10U_20200101-20210101/10U_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32610/10W_20200101-20210101/10W_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32610/10V_20200101-20210101/10V_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32610/10X_20200101-20210101/10X_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32610/catalog.json
esri-lulc-2020-stac esri-lulc-2020/32612/12S_20200101-20210101/12S_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32612/12R_20200101-20210101/12R_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32612/12T_20200101-20210101/12T_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32612/12U_20200101-20210101/12U_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32612/12Q_20200101-20210101/12Q_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32612/12P_2

...

esri-lulc-2020-stac esri-lulc-2020/32752/52J_20200101-20210101/52J_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32752/catalog.json
esri-lulc-2020-stac esri-lulc-2020/32754/54H_20200101-20210101/54H_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32754/54D_20200101-20210101/54D_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32754/54M_20200101-20210101/54M_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32754/54L_20200101-20210101/54L_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32754/54C_20200101-20210101/54C_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32754/54K_20200101-20210101/54K_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32754/54G_20200101-20210101/54G_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32754/54J_20200101-20210101/54J_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32754/catalog.json
esri-lulc-2020-stac esri-lulc-2020/32755/55H_20200101-20210101/55H_20200101-202101

esri-lulc-2020-stac esri-lulc-2020/32628/28X_20200101-20210101/28X_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32628/catalog.json
esri-lulc-2020-stac esri-lulc-2020/32630/30S_20200101-20210101/30S_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32630/30N_20200101-20210101/30N_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32630/30P_20200101-20210101/30P_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32630/30Q_20200101-20210101/30Q_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32630/30R_20200101-20210101/30R_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32630/30T_20200101-20210101/30T_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32630/30V_20200101-20210101/30V_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32630/30U_20200101-20210101/30U_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32630/catalog.json
esri-lulc-2020-stac esri-lulc-2020/32631/31S_20200101-20210101/31S_20200101-202101

esri-lulc-2020-stac esri-lulc-2020/32643/43T_20200101-20210101/43T_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32643/43V_20200101-20210101/43V_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32643/43U_20200101-20210101/43U_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32643/43W_20200101-20210101/43W_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32643/43X_20200101-20210101/43X_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32643/catalog.json
esri-lulc-2020-stac esri-lulc-2020/32645/45S_20200101-20210101/45S_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32645/45Q_20200101-20210101/45Q_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32645/45R_20200101-20210101/45R_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32645/45T_20200101-20210101/45T_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32645/45V_20200101-20210101/45V_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32645/45U_2

esri-lulc-2020-stac esri-lulc-2020/32622/22W_20200101-20210101/22W_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32622/22X_20200101-20210101/22X_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32622/catalog.json
esri-lulc-2020-stac esri-lulc-2020/32601/01W_20200101-20210101/01W_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32601/01U_20200101-20210101/01U_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32601/catalog.json
esri-lulc-2020-stac esri-lulc-2020/32604/04N_20200101-20210101/04N_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32604/04W_20200101-20210101/04W_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32604/04Q_20200101-20210101/04Q_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32604/04V_20200101-20210101/04V_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32604/04U_20200101-20210101/04U_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32604/catalog.json
esri-lulc-2020-stac esri-lul

esri-lulc-2020-stac esri-lulc-2020/32757/57C_20200101-20210101/57C_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32757/57F_20200101-20210101/57F_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32757/catalog.json
esri-lulc-2020-stac esri-lulc-2020/32657/57N_20200101-20210101/57N_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32657/57V_20200101-20210101/57V_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32657/57U_20200101-20210101/57U_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32657/57W_20200101-20210101/57W_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32657/catalog.json
esri-lulc-2020-stac esri-lulc-2020/32758/58D_20200101-20210101/58D_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32758/58L_20200101-20210101/58L_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32758/58C_20200101-20210101/58C_20200101-20210101.json
esri-lulc-2020-stac esri-lulc-2020/32758/58K_20200101-20210101/58K_20200101-202101